In [2]:
# ME 241 Final Project - 1D heat conduction
# Danny Randles, Edgar Rodriguez Marquez, Will Suddeth

# Danny Randles: Designed the matrices for solving the differential equation for temperature. Adjusting the code to include constant, linear, and quadratic functions for 
# the cross sectional area. Applied both 3 point and 5 point methods to solve the differential equation. 

# 

# importing modules for matrix and plotting
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wg

def heat_cond(points,heat_gen,area_type):
    # defining material properties
    LBar   =         10     # length of bar: meters
    kCond  =         100    # thermal conductivity: W/m/C
    hConv  =         10      # convection coefficient: W/m^2/C
    xsInit =         0.1    # initial cross-section area: m^2
    xsFin  =         0.05   # final cross-section area: m^2
    Tair   =         20     # air temp: Celsius
    n      =         10     # number of sections
    sHeat  =         2      # heat gen per unit length: W/m
    sHeatF =         10     # heat gen variable by length: W/m
    # boundary condition
    Tinit  =         300    # initial bar temp: C

    # 3 point, constant area, constant heat gen
    if points == '3' and heat_gen == 'Constant' and area_type == 'Constant':
        dx = LBar/(n-1)
        dx2 = dx*dx

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1
        LM[-1,-1]  = 1

        # for loop
        for i in range(1, n-1):
            LM[i,i-1] =  1
            LM[i,i]   = -2
            LM[i,i+1] =  1


        # defining the last row
        LM[-1,-1] = 1 + ((hConv *dx)/kCond)
        LM[-1,-2] = -1

        # matrix b

        b = np.full(n, ((-sHeat/((kCond)*(xsInit)))*(dx2)))
        b[0]  = 300
        b[-1] = (hConv*dx/kCond)*Tair


        # solve the linear equation
        vectors3pconstant = np.linalg.solve(LM, b)


        xvals = np.linspace(0, 10, n)
        print(f"The temp at x = 10m is {vectors3pconstant[-1].round(3)} Celsius")


        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors3pconstant)
        plt.plot(5, 50)

        plt.title("1D Heat Conduction, Constant Area, Constant Heat Gen, 3 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show()

    # 3 point, linear area, constant heat gen
    elif points == '3' and heat_gen == 'Constant' and area_type == 'Linear':
        #variation of cross sectional area
        # array of steps for area

        # differential lengths
        dx = LBar/(n-1)
        dx2 = dx*dx

        # defining the area function
        def Area(x):
            return 0.05+0.005*(x)
        # defining the derivative of the area function
        Darea = 0.005

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1
        LM[-1,-1]  = 1


        # creating our RHS matrix
        b = np.full(n, 0)

        # for loop for different areas
        for i in range(1, n-1):
            # creating the Left side matrix for rows 2 and beyond
            # including linear x-sectional area function
            LM[i,i-1] =  1*((-(Darea*(dx))/2)  + (Area(i)))
            LM[i,i]   = -2*((Area(i)))
            LM[i,i+1] =  1*(((Darea*(dx))/2)   + (Area(i)))
            # creating our right side matrix
            # sHeat=2
            b[i] = (-sHeat/((kCond)*(dx2)))

        # defining the last row
        LM[-1,-1] = 1 + ((hConv *dx)/kCond)
        LM[-1,-2] = -1

        # matrix b

        b[0]  = 300
        b[1]  = ((sHeat/(kCond))*(dx2))
        b[-2]  = ((sHeat/(kCond))*(dx2))
        b[-1] = (hConv*dx/kCond)*Tair


        # solve the linear equation
        vectors3plineararea = np.linalg.solve(LM, b)

        xvals = np.linspace(0, 10, n)
        print(f"The temp at x = 10m is {vectors3plineararea[-1].round(3)} Celsisus")

        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors3plineararea, 'r-')
        plt.plot(5, 50)
        plt.title("1D Heat Conduction, Linear Area, Constant Heat Gen, 3 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show()

    # 3 point, linear area, linear heat gen
    elif points == '3' and heat_gen == 'Linear' and area_type == 'Linear':
        #variation of cross sectional area
        # array of steps for area

        # differential lengths
        dx = LBar/(n-1)
        dx2 = dx*dx

        # defining the area function with range(0.05,0.1)
        def Area(x):
            return 0.05+0.005*(x)
        # derviative of the area
        Darea = 0.005

        # defining the heat generation function as linear
        # let it range from x(0)=0 to x(10)=10
        def sHeatGen(x):
            return 10 - x

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1



        # creating our RHS matrix
        b = np.full(n, 0)

        # for loop for different areas
        for i in range(1, n-1):
            # creating the Left side matrix for rows 2 and beyond
            # including linear x-sectional area function
            LM[i,i-1] =  1*((-(Darea*(dx))/2)  + (Area(i)))
            LM[i,i]   = -2*((Area(i)))
            LM[i,i+1] =  1*(((Darea*(dx))/2)   + (Area(i)))
            # creating our right side matrix
            # including linear x-sectional area function
            b[i] = (-sHeatGen(i))/(((kCond)*(dx2)))

        # defining the last row
        LM[-1,-1] = 1 + ((hConv *dx)/kCond)
        LM[-1,-2] = -1

        # matrix b

        b[0]  = 300
        b[-1] = (hConv*dx/kCond)*Tair


        # solve the linear equation
        vectors3pLinAreaLinHeatGen = np.linalg.solve(LM, b)

        xvals = np.linspace(0, 10, n)
        print(f"The temp at x = 10m is {vectors3pLinAreaLinHeatGen[-1].round(3)} Celsisus")
        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors3pLinAreaLinHeatGen, '-g')
        plt.plot(5, 50)
        plt.title("1D Heat Conduction, Linear Area, Linear Heat Gen, 3 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show()
    
    # 3 point, quadratic area, linear heat gen
    elif points == '3' and heat_gen == 'Linear' and area_type == 'Quadratic':
        #variation of cross sectional area
        # array of steps for area

        # differential lengths
        dx = LBar/(n-1)
        dx2 = dx*dx

        # defining the area function with range(0.05,0.1)
        def QArea(x):
            return 0.05+(0.0004*x**2)+0.001*x
        # derviative of the area
        def QDarea(x):
            return 0.0008*x +0.001

        # defining the heat generation function as linear
        # let it range from x(0)=0 to x(10)=10
        def sHeatGen(x):
            return x

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1

        # creating our RHS matrix
        b = np.full(n, 0)

        # for loop for different areas
        for i in range(1, n-1):
            # creating the Left side matrix for rows 2 and beyond
            # including linear x-sectional area function
            LM[i,i-1] =  1*((-(QDarea(i)*(dx))/2)  + (QArea(i)))
            LM[i,i]   = -2*((QArea(i)))
            LM[i,i+1] =  1*(((QDarea(i)*(dx))/2)   + (QArea(i)))
            # creating our right side matrix
            # including linear x-sectional area function
            b[i] = (-sHeatGen(i))/(((kCond)*(dx2)))

        # defining the last row
        LM[-1,-1] = 1 + ((hConv *dx)/kCond)
        LM[-1,-2] = -1

        # matrix b

        b[0]  = 300
        b[-1] = (hConv*dx/kCond)*Tair


        # solve the linear equation
        vectors3pQuadAreaLinHeatGen = np.linalg.solve(LM, b)
        print(f"The temp at x = 10m is {vectors3pQuadAreaLinHeatGen[-1].round(3)} Celsius")

        xvals = np.linspace(0, 10, n)

        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors3pQuadAreaLinHeatGen, '-c')
        plt.plot(5, 50)
        plt.title("1D Heat Conduction, Quadratic Area, Linear Heat Gen, 3 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show()

    # 5-point, constant area( A(x)), constant heat gen( s(x))
    elif points == '5' and heat_gen == 'Constant' and area_type == 'Constant':
        dx = LBar/(n-1)
        dx2 = dx*dx

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1
        LM[-1,-1]  = 1


        # defining 2nd row
        LM[1,0]  =  1
        LM[1,1]  = -2
        LM[1,2]  =  1

        # defining 2nd to last row
        LM[-2,-1]  =  1
        LM[-2,-2]  = -2
        LM[-2,-3]  =  1

        # matrix b
        b = np.full(n, ((-sHeat/(kCond))*12*(dx2)))
        b[0]  = 300
        b[-1] = (hConv*dx/kCond)*Tair


        # for loop
        for i in range(2, n-2):
            LM[i,i-2] =  -1
            LM[i,i-1] =  16
            LM[i,i]   = -30
            LM[i,i+1] =  16
            LM[i,i+2] = -1
            # Heat gen linear application
            b[i] = (-sHeat*12*(dx2))/(kCond)

        # defining the last row
        LM[-1,-1] = 1 + (hConv *dx/kCond)
        LM[-1,-2] = -1

        # solve the linear equation
        vectors5pconstant = np.linalg.solve(LM, b)
        print(f"The temp at x = 10m is {vectors5pconstant[-1].round(3)} Celsisus")

        xvals = np.linspace(0, 10, n)
        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors5pconstant, 'sienna')
        plt.plot(5, 50)

        plt.title("1D Heat Conduction, Constant Area, Constant Heat Gen, 5 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show()

    # 5-point, linear area( A(x)), constant heat gen( s(x))
    elif points == '5' and heat_gen == 'Constant' and area_type == 'Linear':
        def Area(x):
            return 0.05+0.005*(x)
        # derviative of the area
        Darea = 0.005

        dx = LBar/(n-1)
        dx2 = dx*dx

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1


        # defining 2nd row
        LM[1,0]  =  1
        LM[1,1]  = -2
        LM[1,2]  =  1

        # defining 2nd to last row
        LM[-2,-1]  =  1
        LM[-2,-2]  = -2
        LM[-2,-3]  =  1

        # matrix b
        b = np.full(n, ((-sHeat/(kCond))*(dx2)))
        b[0]  = 300
        b[-1] = (hConv*dx/kCond)*Tair


        # for loop
        for i in range(2, n-2):
            LM[i,i-2] =  1*((dx*Darea)-Area(i))
            LM[i,i-1] =  (-8*(Darea*dx))+(16*Area(i))
            LM[i,i]   = -30*Area(i)
            LM[i,i+1] =  (8*(Darea*dx))+(16*Area(i))
            LM[i,i+2] = -1*((dx*Darea)+Area(i))
            # Heat gen linear application
            #b[i] = (-sHeatGen(i)*12*(dx2))/(kCond)

        # defining the last row
        LM[-1,-1] = 1 + ((hConv *dx)/kCond)
        LM[-1,-2] = -1


        # solve the linear equation
        vectors5pLinArea = np.linalg.solve(LM, b)
        print(f"The temp at x = 10m is {vectors5pLinArea[-1].round(3)} Celsisus")

        xvals = np.linspace(0, 10, n)
        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors5pLinArea, 'hotpink')
        plt.plot(5, 50)

        plt.title("1D Heat Conduction, Linear Area, Constant Heat Gen, 5 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show()

    # 5-point, Quadratic area( A(x)), constant heat gen( s(x))
    elif points == '5' and heat_gen == 'Constant' and area_type == 'Quadratic':
        # defining the area function with range(0.05,0.1)
        def QArea(x):
            return 0.05+(0.0004*x**2)+0.001*x
        # derviative of the area
        def QDarea(x):
            return 0.0008*x+0.001
        #
        dx = LBar/(n-1)
        dx2 = dx*dx

        # setting up LHS matrix
        LM = np.zeros((n,n))

        # defining initial value
        LM[0,0]  = 1


        # defining 2nd row
        LM[1,0]  =  1
        LM[1,1]  = -2
        LM[1,2]  =  1

        # defining 2nd to last row
        LM[-2,-1]  =  1
        LM[-2,-2]  = -2
        LM[-2,-3]  =  1

        # matrix b
        b = np.full(n, ((-sHeat/(kCond))*(dx2)))
        b[0]  = 300
        b[-1] = (hConv*dx/kCond)*Tair


        # for loop
        for i in range(2, n-2):
            LM[i,i-2] =  1*((dx*QDarea(i))-QArea(i))
            LM[i,i-1] =  (-8*(QDarea(i)*dx))+(16*QArea(i))
            LM[i,i]   = -30*QArea(i)
            LM[i,i+1] =  (8*(QDarea(i)*dx))+(16*QArea(i))
            LM[i,i+2] = -1*((dx*QDarea(i))+QArea(i))
            # Heat gen linear application
            #b[i] = (-sHeatGen(i)*12*(dx2))/(kCond)

        # defining the last row
        LM[-1,-1] = 1 + ((hConv *dx)/kCond)
        LM[-1,-2] = -1


        # solve the linear equation
        vectors5pQuadArea = np.linalg.solve(LM, b)

        print(f"The temp at x = 10m is {vectors5pQuadArea[-1].round(3)} Celsisus")

        xvals = np.linspace(0, 10, n)
        plt.figure(figsize=(10,8))
        plt.plot(xvals, vectors5pQuadArea, 'firebrick')
        plt.plot(5, 50)

        plt.title("1D Heat Conduction, Quadratic Area, Constant Heat Gen, 5 Point Differencing")
        plt.xlabel('Distance from hot end of bar (m)')
        plt.ylabel('Temperature (C)')
        plt.show() 
    else:
        print("Combination Invalid")

point_drop= wg.Select(options=['3','5'], rows=2, description='Points:')
heat_drop= wg.Select(options=['Constant','Linear'],rows=2, description='Heat Gen:')
area_drop= wg.Select(options=['Constant','Linear','Quadratic'],rows=3, description='Area type:')
wg.interact(heat_cond,points=point_drop,heat_gen=heat_drop,area_type=area_drop)

interactive(children=(Select(description='Points:', options=('3', '5'), rows=2, value='3'), Select(description…

<function __main__.heat_cond(points, heat_gen, area_type)>